In [19]:
import pandas as pd
import numpy as np
import duckdb
import warnings
import os

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# email
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import warnings
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']
con = duckdb.connect(database=":memory:")

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

Today's date: 2026-01-07


In [20]:
%run ./common_utils.ipynb

# Initial Functions

In [21]:
def email(model, error):
    
    # Email details
    sender_email = "rodolfoe7157@gmail.com"
    receiver_email = "rodolfoe7157@gmail.com"
    password = "cqgu bfey cnyx sfue"  # See note below

    subject = "NBA create_Predictions error"
    body = f"Model: {model}_model\nERROR: {error}"

    # Create message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Connect to Gmail SMTP server and send
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, password)
        server.send_message(msg)

    print("Email sent successfully!")

In [22]:
def create_base_df():
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    df4 = pd.DataFrame()
    for i in [2022, 2023, 2024, 2025]:
        df_actuals = pd.read_csv(f"../tables/{i}/parlay_stats.csv")
        df_actuals['Season'] = i
        df = pd.concat([df, df_actuals])

        df_schd = pd.read_csv(f"../tables/{i}/nba_schedule.csv")
        df_schd['Season'] = i
        df2 = pd.concat([df2, df_schd])

        df_gms = pd.read_csv(f"../tables/{i}/season_gamelogs.csv")
        df_gms['Season'] = i
        df3 = pd.concat([df3, df_gms])

        df_inj = pd.read_csv(f"../tables/{i}/injuries.csv")
        df_inj['Season'] = i
        df4 = pd.concat([df4, df_inj])

    df['Date'] = pd.to_datetime(df.Date)
    df2['Date'] = pd.to_datetime(df2.Date)
    df3['Date'] = pd.to_datetime(df3.Date)
    df3 = df3[~df3[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df4['Date'] = pd.to_datetime(df4.Date)

    df3_temp = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
    df3_temp['PR'] = df3_temp.PTS + df3_temp.REB 
    df3_temp['PA'] = df3_temp.PTS + df3_temp.AST
    df3_temp['RA'] = df3_temp.REB + df3_temp.AST
    df3_temp['PRA'] = df3_temp.PTS + df3_temp.REB + df3_temp.AST
    df3_temp['STL_BLK'] = df3_temp.STL + df3_temp.BLK
    df = df.merge(df3_temp, on=['Season', 'Date', 'Team', 'Player'], how='left')

    df_mtch = df2[['Season', 'Date', 'AwayABV', 'HomeABV', 'AwayPTS', 'HomePTS', 'AwayB2B', 'HomeB2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch['Team_type'] = 'Away'
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm', 'Team_type']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch2['Team_type'] = 'Home'
    df_mtch = pd.concat([df_mtch, df_mtch2])
    df_mtch = df_mtch[['Season', 'Date', 'Team', 'Team_type', 'AwayPTS', 'HomePTS', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch = df_mtch.sort_values(["Team", "Date"])
    df_mtch['team_game_num'] = df_mtch.groupby(["Team", "Season"]).cumcount() + 1
    df_mtch['Spread'] = np.where(df_mtch.Team_type == 'Home', df_mtch.HomePTS - df_mtch.AwayPTS, df_mtch.AwayPTS - df_mtch.HomePTS)
    df_mtch['Total'] = df_mtch.AwayPTS + df_mtch.HomePTS
    df_mtch['is_Win'] = np.where(df_mtch.Spread > 0, 1, 0)
    df_mtch['Szn_Wins'] = df_mtch.groupby(['Season', 'Team'])['is_Win'].cumsum()
    df = df.drop(['Season', 'Team_type'], axis=1).merge(df_mtch, on=['Date', 'Team'])

    df = df.merge(df4[['Date', 'Team', 'Player', 'Status']], on=['Date', 'Team', 'Player'], how='left')
    df['Status'] = np.where((df.Active == 1) & (df.Status.isnull()), 'Available', df.Status)
    df['Status'] = np.where((df.Active == 0), 'Out', df.Status)
    df['Status'] = np.where((df.Status == 'Out') & (df.Active != 0), 'Available', df.Status)
    
    return df

# Minutes Projection Model

In [23]:
def setup_df_mins(con, df):
    
    df = df[['Season', 'Date', 'Team', 'Team_type', 'Opp', 'Player', 'Pos', 'B2B', 'MP',
             'Spread', 'team_game_num', 'pstszn_gm', 'is_OT']]
    
    for col in ['MP']:
        for N in [1, 3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f'prev_team_mins_pct_L{N}'] = df[f'{col}_L{N}_avg'] / 240

    games_last_7_days = df.sort_values(['Player', 'Season', 'Date']).groupby(['Player', 'Season']).rolling('7D', on='Date', closed='left')['MP'].count().reset_index().rename(columns={"MP": "gms_L7_days"})
    games_last_7_days = games_last_7_days.drop_duplicates(
        subset=['Player', 'Season', 'Date']
    )
    df = df.merge(games_last_7_days, on=['Player', 'Season', 'Date'])
    df['gms_L7_days'] = df.gms_L7_days.fillna(0).astype(int)
        
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for N in [1, 3, 5]:
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df[f'{role}_last{N}'] = (
                df.groupby('Player')[role]
                  .rolling(N, min_periods=1)
                  .sum()
                  .shift(1)
                  .reset_index(0, drop=True)
            )
        rec_role_cols = [f'{role}_last{N}' for role in ['reserve_td', 'bench_td', 'starter_td']]
        df[f'recent_most_common_role_L{N}'] = df[rec_role_cols].idxmax(axis=1)
        df[f'recent_role_L{N}'] = 0
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'starter_td_last{N}', 1, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'bench_td_last{N}', 2, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'reserve_td_last{N}', 3, df[f'recent_role_L{N}'])      
        df = df.drop(f'recent_most_common_role_L{N}', axis=1)
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df = df.drop(f'{role}_last{N}', axis=1)
      
    df['missed_games'] = (
        df.groupby(['Player', 'Team', 'Season'])['team_game_num']      
          .diff()
          .sub(1)
          .fillna(0)
          .astype(int)
    )
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread < 6), 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread >= 6) & abs(df.Spread <= 14)), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread > 14), 3, df.game_spread_type) 

    # Tell model games exist after players injuries/susp
    team_games = df[['Season', 'Team', 'Date', 'team_game_num']].drop_duplicates()
    players = df[['Season','Player','Team']].drop_duplicates()
    fabricated = (players.sort_values('Season').groupby('Player', as_index=False).last())
    fabricated['Season'] = fabricated['Season'] + 1
    players = pd.concat([players, fabricated], ignore_index=True).drop_duplicates(['Season','Player','Team'])
    expanded = team_games.merge(players, on=['Season', 'Team'], how='left')
    expanded = expanded.merge(df[['Season', 'Player', 'Date', 'MP']], on=['Season', 'Player', 'Date'], how='left').drop_duplicates(['Season', 'Date', 'Player','Team'])
    expanded['player_played'] = expanded['MP'].notna().astype(int)
    expanded['team_played_no_player'] = ((expanded['player_played'] == 0)).astype(int)
    expanded['tm_plays_after'] = (expanded.groupby(['Player'])['team_played_no_player'].shift(-1))
    expanded['gms_after'] = 0
    expanded['gms_after'] = np.where((expanded.player_played == 1) & (expanded.tm_plays_after == 1), 1, expanded.gms_after)
    df = df.merge(expanded[['Date', 'Team', 'Player', 'gms_after']], on=['Date', 'Team', 'Player'])
    
    df['MP_change_pct_L10'] = (df['MP_L1_avg'] - df['MP_L10_avg']) / df['MP_L10_avg']
    df['Early_stop'] = (
        (df['MP_L1_avg'] < 5) |  
        ((df.role == 1) & (df.MP_change_pct_L10 <= -0.35)) | 
        ((df.role == 2) & (df.MP_change_pct_L10 <= -0.45)) | 
        ((df.role == 3) & (df.MP_change_pct_L10 <= -0.55))
    ).astype(int)
    df['Early_stop'] = df.groupby('Player')['Early_stop'].shift(-1).fillna(0).astype(int)
    Early_stop_conds = (
                        (((df['MP'] - df['MP_L10_avg']) / df['MP_L10_avg']) <= -0.25) | 
                        ((df.MP < 8) & (df.role != 3))
                       )
    df['Early_stop'] = np.where(Early_stop_conds, 1, df.Early_stop)
    
    df['MP_increase'] = (
        ((df.role == 1) & (df.MP_change_pct_L10 >= 0.15)) |
        ((df.role == 2) & (df.MP_change_pct_L10 >= 0.10)) |
        ((df.role == 3) & (df.MP_change_pct_L10 >= 0.05))
    ).astype(int)
    df['MP_increase'] = df.groupby('Player')['MP_increase'].shift(-1).fillna(0).astype(int)
    MP_Inc_conds = (
                    ((((df['MP'] - df['MP_L10_avg']) / df['MP_L10_avg']) >= 0.15))
                   )
    df['MP_increase'] = np.where(MP_Inc_conds, 1, df.MP_increase)
    df['MP_increase_extreme'] = (
            ((df.MP > df.MP_L5_avg * 3))
    ).astype(int)
    
    df['Injured'] = (
            ((df.MP < df.MP_L5_avg * 0.35) & (df.role != 3) & (df.recent_role_L1 != 3) & (df.recent_role_L3 != 3) & (df.recent_role_L5 != 3) & (df.gms_after > 0)) | 
            ((df.MP < df.MP_L5_avg * 0.25))
    ).astype(int)
    df['return_game'] = ((df.groupby('Player')['Injured'].shift(1) == 1) & (df.missed_games > 0)).astype(int)
    df['games_since_return'] = (df.groupby('Player')['return_game'].cumsum())
    df['games_since_return'] = (df.groupby(['Player', 'games_since_return']).cumcount())
    df['ramp_phase'] = 0
    df.loc[df.return_game == 1, 'ramp_phase'] = 1
    df.loc[df.games_since_return.isin([1, 2, 3]), 'ramp_phase'] = 2
    df.loc[df.games_since_return >= 4, 'ramp_phase'] = 3
    df['starter_return'] = ((df.return_game == 1) & (df.role == 1)).astype(int)
    df['bench_return']   = ((df.return_game == 1) & (df.role == 2)).astype(int)
    
    # Location based features
    df["DaysLstGm"] = (df.groupby("Player")["Date"].diff().dt.days).fillna(0).astype(int)
    df['Location'] = df.apply(lambda r: r['Team'] if r['Team_type'] == 'Home' else r['Opp'], axis=1)
    df['PrevLocation'] = df.groupby('Player')['Location'].shift(1)
    df['same_arena'] = (df['PrevLocation'] == df['Location']).astype(int)

    df = df.drop(['Season', 'Team_type', 'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter', 
                  'PrevLocation', 'Location', 'gms_after', 'return_game', 'MP_change_pct_L10'], axis=1)    
    
    return df

# Main Model

In [24]:
def setup_df_main(df, tgt_stat):
    
    # Stat dependent features 
    if tgt_stat == 'PTS':
        tgt_stat_cols = ['TPM', 'FG', 'FT', 'TPA', 'FGA', 'FTA']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'TPM', 'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        # Efficiency metrics
        df['three_rate_raw'] =  np.where(df.FGA > 0, df['TPA'] / df['FGA'], 0)
        df['ft_rate_raw']    =  np.where(df.FGA > 0, df['FTA'] / df['FGA'], 0)
        df['eFG_raw'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
        df['TS_raw'] = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))    
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        
        for w in [3, 5, 10]:
            for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
                col = f"{metric}_L{w}"
                df[col] = (
                    df.groupby(['Player','Season'])[f'{metric}_raw']
                      .rolling(w, min_periods=1)
                      .mean()
                      .shift(1)
                      .reset_index(level=[0,1], drop=True)
                )
        for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
            col = f'{metric}_weighted'
            df[col] = (
                0.6 * df[f'{metric}_L3'] +
                0.3 * df[f'{metric}_L5'] +
                0.1 * df[f'{metric}_L10']
            )
            df = df.drop(f'{metric}_raw', axis=1)
        
    elif tgt_stat == 'PRA':
        tgt_stat_cols = ['PTS', 'REB', 'AST', 'TPM', 'FG']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK', 
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        for w in [3, 5, 10]:
            df[f"usage_proxy_L{w}"] = (
                df.groupby(['Player','Season'])[f'usage_proxy_raw']
                  .rolling(w, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0,1], drop=True)
            )
        df['usage_proxy_weighted'] = (
            0.6 * df[f'usage_proxy_L3'] +
            0.3 * df[f'usage_proxy_L5'] +
            0.1 * df[f'usage_proxy_L10']
        )
        df = df.drop('usage_proxy_raw', axis=1)
        
        
    else:
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
          f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
          f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]

    
    # Create rolling + lag features    
    for col in ['MP'] + tgt_stat_cols:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [1, 3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )

    # Role identifiers features
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for N in [1, 3, 5]:
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df[f'{role}_last{N}'] = (
                df.sort_values(['Player', 'Date']).groupby('Player')[role]
                  .rolling(N, min_periods=1)
                  .sum()
                  .shift(1)
                  .reset_index(0, drop=True)
            )
        rec_role_cols = [f'{role}_last{N}' for role in ['reserve_td', 'bench_td', 'starter_td']]
        df[f'recent_most_common_role_L{N}'] = df[rec_role_cols].idxmax(axis=1)
        df[f'recent_role_L{N}'] = 0
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'starter_td_last{N}', 1, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'bench_td_last{N}', 2, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'reserve_td_last{N}', 3, df[f'recent_role_L{N}'])      
        df = df.drop(f'recent_most_common_role_L{N}', axis=1)
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df = df.drop(f'{role}_last{N}', axis=1)
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread < 6), 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread >= 6) & abs(df.Spread <= 14)), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread > 14), 3, df.game_spread_type) 
    
    for col in categories + ['Season', 'FG', 'FGA', 'FT', 'FTA', 'TPM', 'TPA', 
                             'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter'] + tgt_stat_cols:
        if col == tgt_stat:
            continue
        if col in df.columns:
            df = df.drop(col, axis=1)
        
    return df

### Today's predictions

In [25]:
def generate_predictions(tgt_stat):
    team_encoder = LabelEncoder()
    player_encoder = LabelEncoder()
    team_type_encoder = LabelEncoder()
    position_encoder = LabelEncoder()
    status_encoder = LabelEncoder()
    
    df_pred = create_base_df()
    
    # Encode string cols
    team_encoder.fit(pd.concat([df_pred["Team"], df_pred["Opp"]], axis=0))
    df_pred["Team"] = team_encoder.transform(df_pred["Team"])
    df_pred["Opp"] = team_encoder.transform(df_pred["Opp"])
    df_pred["Player"] = player_encoder.fit_transform(df_pred["Player"])
    df_pred["Pos"] = position_encoder.fit_transform(df_pred["Pos"])
    df_pred['Team_type'] = team_type_encoder.fit_transform(df_pred['Team_type'])
    df_pred["Status"] = status_encoder.fit_transform(df_pred["Status"])
    
    mins_model = xgb.XGBRegressor()
    mins_model.load_model("../ML_models/mins_model.json")
    stat_model = xgb.XGBRegressor()
    stat_model.load_model(f"../ML_models/{tgt_stat}_model.json")
    
    df_lines = pd.read_csv(f"../tables/2025/parlay_lines.csv")
    df_lines['Date'] = pd.to_datetime(df_lines.Date)
    df_lines = df_lines[~(df_lines.Team.isnull())]

    # Predict Mins
    df_lines["Team"] = team_encoder.transform(df_lines["Team"])
    df_pred = df_pred.merge(df_lines[['Date', 'Team', 'Spread', 'Total']], on=['Date', 'Team'], how='left')
    df_pred = df_pred[~df_pred[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
    df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
    df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
    df_pred_mins = setup_df_mins(con, df_pred)
    df_pred_mins = df_pred_mins.drop(['Date', 'MP'], axis=1)
    df_pred['MP'] = mins_model.predict(df_pred_mins)

    # Predict Stat
    df_pred = setup_df_main(df_pred, tgt_stat)
    feature_cols = [col for col in df_pred.columns if col not in ['Date', tgt_stat]]
    df_pred = df_pred[df_pred.Date == now][feature_cols]
    df_pred[f"{tgt_stat}_proj"] = stat_model.predict(df_pred)

    df_pred['Team'] = team_encoder.inverse_transform(df_pred["Team"])
    df_lines['Team'] = team_encoder.inverse_transform(df_lines["Team"])
    df_pred['Opp'] = team_encoder.inverse_transform(df_pred["Opp"])
    df_pred['Player'] = player_encoder.inverse_transform(df_pred["Player"])
    df_pred['Pos'] = position_encoder.inverse_transform(df_pred["Pos"])

    df_lines = df_lines[df_lines.Date == now][['Team', 'Player', f'{tgt_stat}_line']]
    df_pred = df_pred.merge(df_lines, on=['Team', 'Player'])

    tds_picks = df_pred[~(df_pred[f'{tgt_stat}_line'].isnull())]\
                [['Team', 'Player', 'Pos', 'Opp', 'MP', 'MP_L5_avg', f'{tgt_stat}_line', f'{tgt_stat}_proj']]
    tds_picks['Diff'] = abs((df_pred[f'{tgt_stat}_line'] - df_pred[f'{tgt_stat}_proj']))
    tds_picks['Diff2'] = abs((df_pred['MP'] - df_pred['MP_L5_avg']))
    tds_picks = tds_picks.sort_values('Diff', ascending=False).drop(['Diff', 'Diff2'], axis=1)
    if tds_picks.shape[0] >= 50:
        print(tds_picks.shape[0], 'rows')
        for tm in tds_picks.Team.unique():
            display(tds_picks[tds_picks.Team == tm])
    else:
        display(tds_picks)
    tds_picks.insert(0, 'Date', pd.to_datetime(now))
    partition_save_df(tds_picks, f"../tables/2025/gmday_preds_{tgt_stat}.csv")

In [26]:
try: 
    generate_predictions('PTS')
except Exception as e:
    email('PTS', e)
    raise Exception(e)

151 rows


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
10,UTA,Keyonte George,PG,OKC,33.543938,35.166566,21.5,29.679403
98,UTA,Lauri Markkanen,PF,OKC,33.379692,35.302673,23.5,28.588001
137,UTA,Jusuf Nurkic,C,OKC,26.670835,29.590232,10.5,15.073216
88,UTA,Brice Sensabaugh,SF,OKC,24.729319,28.391840,10.5,14.257651
25,UTA,Isaiah Collier,PG,OKC,24.906584,24.856380,7.5,10.914994
86,UTA,Ace Bailey,SF,OKC,24.126925,26.266035,8.5,9.708200
90,UTA,Svi Mykhailiuk,SF,OKC,17.991518,17.690666,7.5,6.525830


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
140,DET,Paul Reed,C,CHI,12.005345,12.505324,12.5,5.550787
128,DET,Isaiah Stewart,C,CHI,24.929955,25.484725,13.5,7.612842
78,DET,Ausar Thompson,SF,CHI,22.595516,24.368644,13.5,8.823014
75,DET,Duncan Robinson,SF,CHI,24.403109,26.392405,11.5,7.701753
0,DET,Cade Cunningham,PG,CHI,35.323189,34.611742,27.5,27.116268


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
82,DEN,Peyton Watson,SF,BOS,35.317657,35.434296,14.5,20.343151
49,DEN,Tim Hardaway Jr.,SG,BOS,30.529261,30.392149,12.5,17.880157
56,DEN,Bruce Brown,SG,BOS,27.433073,26.692981,5.5,8.054280
53,DEN,Christian Braun,SG,BOS,25.713593,25.633774,9.5,10.665285
100,DEN,Aaron Gordon,PF,BOS,23.199917,21.389160,14.5,15.286947
7,DEN,Jamal Murray,PG,BOS,37.358360,35.877157,26.5,27.040377


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
30,PHI,VJ Edgecombe,SG,WAS,35.927406,36.339314,16.5,22.137396
3,PHI,Tyrese Maxey,PG,WAS,37.592789,39.591845,28.5,31.103931
31,PHI,Quentin Grimes,SG,WAS,31.352818,33.913608,11.5,13.426015
81,PHI,Kelly Oubre Jr.,SF,WAS,30.695213,33.559907,10.5,12.362174
96,PHI,Paul George,PF,WAS,33.019917,33.557655,14.5,15.010700
120,PHI,Joel Embiid,C,WAS,33.240356,35.497477,26.5,26.583101


...

,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
24,ORL,Anthony Black,PG,BRK,31.886951,31.867728,18.5,16.227232
58,ORL,Jase Richardson,SG,BRK,17.320728,16.639292,8.5,6.813137
145,ORL,Goga Bitadze,C,BRK,15.926118,18.375539,5.5,4.270159
95,ORL,Paolo Banchero,PF,BRK,35.079052,33.417446,23.5,23.953201
35,ORL,Desmond Bane,SG,BRK,34.132778,33.682843,20.5,20.082560
131,ORL,Wendell Carter Jr.,C,BRK,29.937626,29.925401,12.5,12.534018


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
110,HOU,Tari Eason,PF,POR,22.628149,25.042921,14.5,12.252751
141,HOU,Steven Adams,C,POR,19.678379,22.395846,7.5,6.321048
99,HOU,Jabari Smith Jr.,PF,POR,36.281364,32.887231,14.5,15.490479
149,HOU,Clint Capela,C,POR,9.856972,13.939766,4.5,3.795204
79,HOU,Amen Thompson,SF,POR,36.720459,35.524719,18.5,17.841803
15,HOU,Reed Sheppard,PG,POR,26.317446,29.499759,11.5,11.194818
69,HOU,Kevin Durant,SF,POR,36.944111,35.606481,27.5,27.265676


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
45,GSW,Brandin Podziemski,SG,MIL,25.616362,25.808388,10.5,8.348938
27,GSW,De'Anthony Melton,PG,MIL,19.770430,25.253004,9.5,7.518318
72,GSW,Jimmy Butler,SF,MIL,33.602821,32.556734,19.5,17.544758
11,GSW,Stephen Curry,PG,MIL,34.431202,33.089830,28.5,26.941702
147,GSW,Al Horford,C,MIL,15.867138,14.598037,5.5,6.936516
114,GSW,Quinten Post,PF,MIL,17.012150,16.126849,6.5,7.740764
47,GSW,Moses Moody,SG,MIL,22.318659,22.462987,8.5,7.427284
112,GSW,Draymond Green,PF,MIL,23.730658,24.695327,8.5,9.294888
57,GSW,Gary Payton II,SG,MIL,12.615602,15.344443,4.5,5.113376


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
12,MEM,Ja Morant,PG,PHO,25.563351,29.586655,19.5,17.513096
127,MEM,Jaren Jackson Jr.,C,PHO,33.834808,34.767117,19.5,20.826569
104,MEM,Santi Aldama,PF,PHO,31.155096,33.077087,14.5,15.683333
36,MEM,Jaylen Wells,SG,PHO,29.456530,30.004521,13.5,14.294945
139,MEM,Jock Landale,C,PHO,20.229855,18.053783,12.5,12.423742


../tables/2025/gmday_preds_PTS.csv saved!


In [27]:
try: 
    generate_predictions('PRA')
except Exception as e:
    email('PRA', e)
    raise Exception(e)

101 rows


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
140,DET,Paul Reed,C,CHI,12.005345,12.505324,23.5,12.385221
78,DET,Ausar Thompson,SF,CHI,22.595516,24.368644,23.5,15.832730
128,DET,Isaiah Stewart,C,CHI,24.929955,25.484725,22.5,15.161065
0,DET,Cade Cunningham,PG,CHI,35.323189,34.611742,44.5,40.449505
75,DET,Duncan Robinson,SF,CHI,24.403109,26.392405,16.5,13.108965


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
2,OKC,Shai Gilgeous-Alexander,PG,UTA,30.979288,31.084620,42.5,33.418209
29,OKC,Jalen Williams,SG,UTA,28.521605,28.045473,32.5,24.784960


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
97,NOP,Zion Williamson,PF,ATL,27.445936,29.085630,35.5,28.031504
64,NOP,Trey Murphy III,SF,ATL,36.078587,34.683667,29.5,34.345127
19,NOP,Jeremiah Fears,PG,ATL,23.795605,25.604487,21.5,19.373589
126,NOP,Derik Queen,C,ATL,27.676792,27.489482,27.5,25.993727


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
10,UTA,Keyonte George,PG,OKC,33.543938,35.166566,31.5,38.553398
137,UTA,Jusuf Nurkic,C,OKC,26.670835,29.590232,24.5,30.546171
98,UTA,Lauri Markkanen,PF,OKC,33.379692,35.302673,31.5,36.083992


...

,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
24,ORL,Anthony Black,PG,BRK,31.886951,31.867728,27.5,24.953943
35,ORL,Desmond Bane,SG,BRK,34.132778,33.682843,29.5,30.179457
95,ORL,Paolo Banchero,PF,BRK,35.079052,33.417446,37.5,37.529060


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
15,HOU,Reed Sheppard,PG,POR,26.317446,29.499759,18.5,15.988242
69,HOU,Kevin Durant,SF,POR,36.944111,35.606481,37.5,38.394081
99,HOU,Jabari Smith Jr.,PF,POR,36.281364,32.887231,24.5,24.978437
79,HOU,Amen Thompson,SF,POR,36.720459,35.524719,31.5,31.386499


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
55,BRK,Terance Mann,SG,ORL,25.970457,26.425026,13.5,15.980310
118,BRK,Danny Wolf,PF,ORL,21.036524,21.153485,12.5,14.369824
62,BRK,Michael Porter Jr.,SF,ORL,34.071156,32.900061,35.5,36.245319


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
46,WAS,Bilal Coulibaly,SG,PHI,26.592480,27.773218,19.5,20.809727
40,WAS,Tre Johnson,SG,PHI,22.916563,26.405395,19.5,19.810049


../tables/2025/gmday_preds_PRA.csv saved!
